<a href="https://colab.research.google.com/github/MarinaStepanova1709/Spark_sql/blob/main/HW_Spark_SQL_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Установка pyspark с помощью pip

In [47]:
pip install --quiet pyspark

In [54]:
from pyspark.sql import SparkSession

# остановить sparkSession
# spark.stop()
spark = SparkSession.builder.appName("Practice").getOrCreate()

# Чтение из файла

In [55]:
from pyspark import SparkFiles

# covid_data_file_url = "https://raw.githubusercontent.com/glincow/netology-spark-sql/main/data/covid-data.csv"
covid_data_file_url = "https://raw.githubusercontent.com/MarinaStepanova1709/Spark_sql/main/covid-data.csv"
spark.sparkContext.addFile(covid_data_file_url)
file_path  = 'file://' + SparkFiles.get('covid-data.csv')
df = spark.read.option('inferSchema', 'true').option('header', 'true').csv(file_path)

Посмотрим, что в таблице


In [58]:
df.show(15)


+--------+---------+-----------+----------+-----------+---------+------------------+------------+----------+-------------------+-----------------------+---------------------+------------------------------+------------------------+----------------------+-------------------------------+-----------------+------------+------------------------+-------------+-------------------------+---------------------+---------------------------------+----------------------+----------------------------------+---------+-----------+------------------------+----------------------+------------------+-------------------------------+-------------+--------------+-----------+------------------+-----------------+-----------------------+----------------+-------------------------+------------------------------+-----------------------------+-----------------------------------+-------------------------------------+----------------+-----------+------------------+----------+-------------+-------------+--------------+--

In [ ]:
# df.schema.json()

In [59]:
# df.printSchema()

1. Выберите 15 стран с наибольшим процентом переболевших на 31 марта (в выходящем датасете необходимы колонки: iso_code, страна, процент переболевших)

In [61]:
from pyspark.sql import functions as F
# выбираем колонки 'iso_code', 'location' и процент переболевших = (всего случаев)/ население
# условие  ('date') == '2020-03-31'
df_cases = df.select('iso_code', 'location'
                     , (F.col('total_cases')/F.col('population') *100).alias('total_cases_percentage')
).where(F.col('date') == '2020-03-31')

# отсортируем по убыванию случаев в процентном выражении (total_cases_percentage) и оставим 15 верхних
df_cases.sort((F.col('total_cases_percentage')).desc()).limit(15).show()


+--------+--------------+----------------------+
|iso_code|      location|total_cases_percentage|
+--------+--------------+----------------------+
|     VAT|       Vatican|    0.7416563658838072|
|     SMR|    San Marino|    0.6953857033413873|
|     AND|       Andorra|    0.4866368989840161|
|     LUX|    Luxembourg|    0.3479366621084514|
|     ISL|       Iceland|    0.3326007326007326|
|     ESP|         Spain|   0.20516189755388237|
|     CHE|   Switzerland|   0.19186288753587968|
|     LIE| Liechtenstein|   0.17830453365498072|
|     ITA|         Italy|    0.1749732078891164|
|     MCO|        Monaco|   0.13250433187238814|
|     AUT|       Austria|   0.11303073370047965|
|     BEL|       Belgium|   0.11022798339479065|
|OWID_EUN|European Union|   0.08938996679530879|
|     DEU|       Germany|   0.08570615766540952|
|     NOR|        Norway|   0.08560768916052816|
+--------+--------------+----------------------+



### 2. Top 10 стран с максимальным зафиксированным кол-вом новых случаев за последнюю неделю марта 2021 в отсортированном порядке по убыванию
(в выходящем датасете необходимы колонки: число, страна, кол-во новых случаев)

In [62]:
start_date = '2021-03-25'
end_date = '2021-03-31'

# выберем даты между start_date и end_date
df_new_cases = df.where(F.col('date').between(start_date,end_date))
# Группируем по 'location' и находим максимум new_cases за неделю
# потом максимум делим на население
df_sum_by_location = df_new_cases.groupBy('iso_code','location') \
    .agg(
        F.max('new_cases').alias('new_cases_max')
    )

# df_sum_by_location.sort((F.col('new_cases_max')).desc()).limit(10).show()

# Из результата исключим строки с "OWID" - суммарные показатели
df_sum_by_location.where(~F.col('iso_code').contains('OWID')).select('iso_code', 'location', 'new_cases_max')\
.sort((F.col('new_cases_max')).desc()).limit(10).show()


+--------+-------------+-------------+
|iso_code|     location|new_cases_max|
+--------+-------------+-------------+
|     BRA|       Brazil|     100158.0|
|     USA|United States|      77321.0|
|     IND|        India|      72330.0|
|     FRA|       France|      59054.0|
|     TUR|       Turkey|      39302.0|
|     POL|       Poland|      35145.0|
|     DEU|      Germany|      25014.0|
|     ITA|        Italy|      24076.0|
|     PER|         Peru|      19206.0|
|     UKR|      Ukraine|      18226.0|
+--------+-------------+-------------+




3. Посчитайте изменение случаев относительно предыдущего дня в России за последнюю неделю марта 2021. (например: в россии вчера было 9150 , сегодня 8763, итог: -387) (в выходящем датасете необходимы колонки: число, кол-во новых случаев вчера, кол-во новых случаев сегодня, дельта)

In [63]:
# выберем из датасета Россию за даты между '2021-03-24','2021-03-31'
df_russia = df.select('iso_code', 'location', 'date', 'new_cases') \
  .where(
    (F.col('location').startswith("Rus")) &
    F.col('date').between('2021-03-24','2021-03-31'))
# df_russia.show()

In [ ]:
# импортируем оконные функции
from pyspark.sql import Window
# внутри окна только сортировка по дате по возрастанию, без партишена
# сначала добавим 'prev_day_new_cases' через lag
# потом вычислим разницу между сегодня и вчера = delta
w = Window.orderBy(('date'))
# lag - предыдущее значение
df_russia.select(
    'iso_code',
    'date',
    F.lag('new_cases').over(w).alias('prev_day_new_cases'),
    'new_cases'
)\
.withColumn('delta', F.col('new_cases') - F.col('prev_day_new_cases'))\
.where( F.col('date').between('2021-03-25','2021-03-31'))\
.show()

+--------+----------+------------------+---------+------+
|iso_code|      date|prev_day_new_cases|new_cases| delta|
+--------+----------+------------------+---------+------+
|     RUS|2021-03-25|            8769.0|   9128.0| 359.0|
|     RUS|2021-03-26|            9128.0|   9073.0| -55.0|
|     RUS|2021-03-27|            9073.0|   8783.0|-290.0|
|     RUS|2021-03-28|            8783.0|   8979.0| 196.0|
|     RUS|2021-03-29|            8979.0|   8589.0|-390.0|
|     RUS|2021-03-30|            8589.0|   8162.0|-427.0|
|     RUS|2021-03-31|            8162.0|   8156.0|  -6.0|
+--------+----------+------------------+---------+------+

